Analysing the Medicaid and Chip Eligibilty

In [ ]:
Import pandas Libriries

In [1]:
import pandas as pd

In [ ]:
#convert the Data Dictionary csv file into .md file

In [56]:
Data_Dictionary = pd.read_csv("../data/Data_Dictionary.csv")
Data_Dictionary = Data_Dictionary.drop(columns=['name','type','format'])
#Convert the data frame into a mark down string(Github style)
Data_Dictionary = Data_Dictionary.to_markdown(tablefmt="github", index=False)
# Export the Markdown string to a .md file
output_path = "../Data_Dictionary.md"
with open(output_path, 'w', encoding='utf-8') as f:
  f.write(Data_Dictionary)

In [8]:
# Read the csv data into medicaid dataframe
medicaid = pd.read_csv("../data/medicaid.csv")
medicaid

,State Abbreviation,State Name,Reporting Period,State Expanded Medicaid,Preliminary or Updated,Final Report,New Applications Submitted to Medicaid and CHIP Agencies,New Applications Submitted to Medicaid and CHIP Agencies - footnotes,Applications for Financial Assistance Submitted to the State Based Marketplace,Applications for Financial Assistance Submitted to the State Based Marketplace - footnotes,...,Total Medicaid and CHIP Determinations Processed between 31 days and 45 days,Total Medicaid and CHIP Determinations Processed between 31 days and 45 days - footnotes,Total Medicaid and CHIP Determinations Processed in More than 45 Days,Total Medicaid and CHIP Determinations Processed in More than 45 Days - footnotes,Total Call Center Volume (Number of Calls),Total Call Center Volume (Number of Calls) - footnotes,Average Call Center Wait Time (Minutes),Average Call Center Wait Time (Minutes) - footnotes,Average Call Center Abandonment Rate,Average Call Center Abandonment Rate - footnotes
0,AK,Alaska,201309,N,U,Y,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Alaska,201706,Y,P,N,3017.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,Alaska,201706,Y,U,Y,3069.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,Alaska,201707,Y,P,N,3187.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,Alaska,201707,Y,U,Y,3265.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,WY,Wyoming,202508,N,P,N,1396.0,Includes Administrative Data Transfers; Does N...,0.0,NaN,...,88.0,NaN,306.0,NaN,11046.0,NaN,1.0,Call centers offer callbacks,0.003,Callbacks are included
10298,WY,Wyoming,202508,N,U,Y,1396.0,Includes Administrative Data Transfers; Does N...,0.0,NaN,...,88.0,NaN,306.0,NaN,11046.0,NaN,1.0,Call centers offer callbacks,0.003,Callbacks are included
10299,WY,Wyoming,202509,N,P,N,1541.0,Includes Administrative Data Transfers; Does N...,0.0,NaN,...,114.0,NaN,175.0,NaN,12548.0,NaN,1.0,Call centers offer callbacks,0.004,Callbacks are included
10300,WY,Wyoming,202509,N,U,Y,1541.0,Includes Administrative Data Transfers; Does N...,0.0,NaN,...,114.0,NaN,175.0,NaN,12548.0,NaN,1.0,Call centers offer callbacks,0.004,Callbacks are included


Clean the Data

In [5]:
medicaid.columns

Index(['State Abbreviation', 'State Name', 'Reporting Period',
       'State Expanded Medicaid', 'Preliminary or Updated', 'Final Report',
       'New Applications Submitted to Medicaid and CHIP Agencies',
       'New Applications Submitted to Medicaid and CHIP Agencies - footnotes',
       'Applications for Financial Assistance Submitted to the State Based Marketplace',
       'Applications for Financial Assistance Submitted to the State Based Marketplace - footnotes',
       'Total Applications for Financial Assistance Submitted at State Level',
       'Total Applications for Financial Assistance Submitted at State Level - footnotes',
       'Individuals Determined Eligible for Medicaid at Application',
       'Individuals Determined Eligible for Medicaid at Application - footnotes',
       'Individuals Determined Eligible for CHIP at Application',
       'Individuals Determined Eligible for CHIP at Application - footnotes',
       'Total Medicaid and CHIP Determinations',
       'To

In [26]:
medicaid['Individuals Determined Eligible for Medicaid at Application - footnotes'].unique()

array([nan, 'Includes Renewals and/or Redeterminations; Includes CHIP',
       'Does Not Include All Medicaid Determinations Made At Application',
       'Includes Renewals and/or Redeterminations; Does Not Include All Medicaid Determinations Made At Application',
       'Includes Renewals and/or Redeterminations',
       'Does Not Include All Determinations for Limited-Benefit Programs Made At Application',
       'Count is of Households, Not Individuals; Includes Renewals and/or Redeterminations',
       'Does Not Include All MAGI Determinations Made At Application; Includes Renewals and/or Redeterminations',
       'Includes Renewals and/or Redeterminations; Does Not Include All MAGI Determinations Made At Application',
       'Includes Final Eligibility Determinations Made by FFM; Includes Conditional Eligibility Determinations; Includes Presumptive Eligibility Determinations',
       'Includes Conditional Eligibility Determinations; Includes Presumptive Eligibility Determinations;

In [ ]:
df.drop(columns=['column_nameA', 'column_nameB'])

In [28]:
medicaid['Applications for Financial Assistance Submitted to the State Based Marketplace'].nunique()

1373